In [48]:
import re,yaml,codecs
from os.path import expanduser
import pandas as pd
from cellbell import ding

In [49]:
home = expanduser("~")
annee="2020"
numGroupe=50
repertoire=home+"/ownCloud/Cours/Bordeaux/L1-UE4-Morphologie/00-Kanonik/"+annee
serie=repertoire+"/K%02d/"%numGroupe

In [50]:
nomLexique=home+"/ownCloud/Cours/Bordeaux/L2-XML/XML-Ressources/bdlexique.txt"
dfLexique=pd.read_csv(nomLexique,encoding="utf8",sep=";")

In [51]:
dfLexique.columns="ortho phono liaison cat ms tam lemme L23 F1 F2 F3 F4".split()
catLexique={"NOM":"N G".split(),"VER":"V".split(),"ADJ":"F G J K L M".split()}
tam="pi ii ai".split()
pers="3S 3P".split()

In [52]:
#dfLexique#[dfLexique["lemme"]=="coyote"]["ortho"].tolist()

In [53]:
newStems={}
with open(serie+"Stems.csv", 'r') as stream:
    stems=yaml.safe_load(stream)
stems

IOError: [Errno 2] No such file or directory: '/Users/gilles/ownCloud/Cours/Bordeaux/L1-UE4-Morphologie/00-Kanonik/2020/K50/Stems.csv'

In [ ]:
def getFormesVerbe(verbe):
    p3S=dfLexique.loc[dfLexique["cat"].isin(catLexique["VER"]) 
                          & (dfLexique["lemme"]==verbe)
                          & (dfLexique["tam"]=="pi")
                          & (dfLexique["ms"]=="3S"), "ortho"].tolist()[0]
    p3P=dfLexique.loc[dfLexique["cat"].isin(catLexique["VER"]) 
                          & (dfLexique["lemme"]==verbe)
                          & (dfLexique["tam"]=="pi")
                          & (dfLexique["ms"]=="3P"), "ortho"].tolist()[0]
    i3S=dfLexique.loc[dfLexique["cat"].isin(catLexique["VER"]) 
                          & (dfLexique["lemme"]==verbe)
                          & (dfLexique["tam"]=="ii")
                          & (dfLexique["ms"]=="3S"), "ortho"].tolist()[0]
    i3P=dfLexique.loc[dfLexique["cat"].isin(catLexique["VER"]) 
                          & (dfLexique["lemme"]==verbe)
                          & (dfLexique["tam"]=="ii")
                          & (dfLexique["ms"]=="3P"), "ortho"].tolist()[0]
    a3S=dfLexique.loc[dfLexique["cat"].isin(catLexique["VER"]) 
                          & (dfLexique["lemme"]==verbe)
                          & (dfLexique["tam"]=="ai")
                          & (dfLexique["ms"]=="3S"), "ortho"].tolist()[0]
    a3P=dfLexique.loc[dfLexique["cat"].isin(catLexique["VER"]) 
                          & (dfLexique["lemme"]==verbe)
                          & (dfLexique["tam"]=="ai")
                          & (dfLexique["ms"]=="3P"), "ortho"].tolist()[0]
    
    return p3S,p3P,i3S,i3P,a3S,a3P
                             
def getFormes(lemme,cat):
    if cat=="VER":
        flexion=list(getFormesVerbe(lemme))
    elif cat=="DET":
        if lemme=="DEF": flexion="le la les l".split()
        elif lemme=="IND": flexion="un une des dE d".split()
        else: flexion="ce cet cette ces".split()
    else:
        flexion=dfLexique.loc[dfLexique["cat"].isin(catLexique[cat]) & (dfLexique["lemme"]==lemme), "ortho"].tolist()
        flexion=list(set(flexion))
        if cat=="NOM" and lemme.endswith("s"):
            print "attention",lemme,"a été modifié avec un S"
            lemme=lemme[:-1]+"S"
        if lemme in flexion:
            flexion.remove(lemme)
    return [lemme]+flexion

In [ ]:
apiSymboles=u"ʃʒŋθð"
sampaSymboles=u"SZNTD"
translit={}
for i,s in enumerate(apiSymboles):
    translit[s]=sampaSymboles[i]
translit

In [ ]:
def getTranslit(chaine):
    result=chaine
    for s in translit:
        result=result.replace(s,translit[s])
    return result

In [ ]:
getTranslit(u"ʃaʒuŋoθið")

In [ ]:
for cat in stems:
    if cat not in newStems:
        newStems[cat]={}
    if cat not in ["PREP"]:
        print cat
        for element in stems[cat]:
            if isinstance(stems[cat][element],dict):
                newStems[cat][element]={}
                for word in stems[cat][element]:
                    if isinstance(stems[cat][element][word],dict):
                        subelement=word
                        if subelement not in newStems[cat][element]:
                            newStems[cat][element][subelement]={}
                        for w in stems[cat][element][subelement]:
                            mot=getTranslit(w)
                            flexion=stems[cat][element][subelement][w]
                            lemme=flexion[0]
                            print getFormes(lemme,cat)
                            newStems[cat][element][subelement][mot]=getFormes(lemme,cat)                            
                    else:
                        mot=getTranslit(word)
                        flexion=stems[cat][element][word]
                        lemme=flexion[0]
                        print lemme, cat
                        print getFormes(lemme,cat)
                        newStems[cat][element][mot]=getFormes(lemme,cat)
                    
            else:
                flexion=stems[cat][element]
                lemme=flexion[0]
                print lemme
                print getFormes(lemme,cat)
                mot=getTranslit(element)
                newStems[cat][mot]=getFormes(lemme,cat)        
    else:
        print cat
        for element in stems[cat]:
            mot=getTranslit(element)
            newStems[cat][mot]=stems[cat][element]

In [ ]:
newStems

In [ ]:
with open(serie+"Stems.yaml", 'w') as stream:
    yaml.safe_dump(newStems,stream=stream,default_flow_style=False,allow_unicode=True)

In [ ]:
ding()